### Regular Expressions

In [1]:
from dexter.DB import DB, RegExp, Action, Entry
import re

In [2]:
DB.open('dev')

In [3]:
e = list(RegExp.objects)

In [4]:
e

[<RegExp: <RE xfer Xfer (to|from) .*?\d+ (.*?)/ Transfer {1} $>>,
 <RegExp: <RE xfer Transfer (to|from) (.*?)/Xfer(.*) Transfer {1} $>>,
 <RegExp: <RE trans .*ABEBOOKS AbeBooks $JC>>,
 <RegExp: <RE trans .*ATLANTIC MNTLY The Atlantic $JC>>,
 <RegExp: <RE trans .*COMCAST Comcast $utility>>,
 <RegExp: <RE trans .*DDA TO DDA Wells Fargo $utility>>,
 <RegExp: <RE trans .*EL METATE El Metate $food:groceries>>,
 <RegExp: <RE trans .*HEY NEIGHBOR Hey Neighbor $food:restaurant>>,
 <RegExp: <RE trans .*Home Depot Home Depot $home>>,
 <RegExp: <RE trans .*HOME DEPOT Home Depot $home>>,
 <RegExp: <RE trans .*LIQUOR STORE (\d+) Liquor Store #{0} $JC>>,
 <RegExp: <RE trans .*MARKET OF CHOICE Market of Choice $food:groceries>>,
 <RegExp: <RE trans .*New York Times New York Times $shared>>,
 <RegExp: <RE trans .*OPB Member OPB $gift:donation>>,
 <RegExp: <RE trans .*PERS BENEFITS PERS $PERS>>,
 <RegExp: <RE trans .*PROVISIONS SOUT Provisions South $JC>>,
 <RegExp: <RE trans .*SSA TREAS Social Securit

In [5]:
RegExp.objects[0]

<RegExp: <RE xfer Xfer (to|from) .*?\d+ (.*?)/ Transfer {1} $>>

In [6]:
e[0].apply('CHEVRON 0092601')

In [7]:
e[2].apply('Payment to Comcast')

In [8]:
e[3].apply('Check # 161: Completed')

In [9]:
e[4].apply('WSFERRIES-ANACORTES')

In [10]:
bool(re.match(e[0].expr,'CHEVRON 0092601',re.I))

False

In [11]:
e[0].matches('CHEVRON')

False

In [12]:
e = DB.find_regexp('CHEVRON')[0]

In [13]:
e.action == Action.T

True

In [14]:
e.expr

'CHEVRON'

In [15]:
e.repl

'Chevron'

In [16]:
e.acct

'car'

In [17]:
DB.find_regexp('C')

[]

### Transfers

Experiment with patterns to use in the `pair` script.  

OCCU is all over the place.
* most transfers have "Xfer (to|from)" in the description field
* transfers scheduled through bill pay have "Transfer (to|from)" in the description field but "Xfer (to|from)" in the extended description
* some have both forms!  e.g. BMW payment is "Xfer to" and "Transfer from"
* Venmo, PayPal, are like bill pay but the extended description has different format (but does start with "Xfer")

Solution: use separate rules for the two forms.

In [18]:
# OCCU transfers
elist = [
    r'Xfer (to|from) .*?\d+ (.*?)/',
    r'Transfer (to|from) (.*?)/Xfer(.*)',
]

In [ ]:
# Card payments, OCCU side
elist = [
    r'Payment to (Chase|Citi|Apple)',
]

In [ ]:
# Card payments
elist = [
    r'AUTOMATIC PAYMENT',   # Chase cards
]

In [33]:
for e in DB.select(Entry, tag='#unpaired'):
    for i, expr in enumerate(elist):
        if m := re.match(expr, e.description, re.I):
            print(i, e.account, e.date, e.amount, e.column, e.description)
            print(m.groups())
            continue

0 chase:freedom 2025-01-03 3.0 debit AUTOMATIC PAYMENT - THANK
()
0 chase:sapphire-jc 2025-01-09 1450.12 debit AUTOMATIC PAYMENT - THANK
()
0 chase:sapphire-lc 2025-01-07 425.33 debit AUTOMATIC PAYMENT - THANK
()
0 chase:united 2025-01-12 957.05 debit AUTOMATIC PAYMENT - THANK
()
